In [ ]:
# Module imports
import text_summarization as ts
import header_extraction as he
import image_generation as ig
import image_searching as ss
import read_file as rf

# PowerPoint imports
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor
from pptx.oxml.xmlchemy import OxmlElement

In [ ]:
text_string = rf.read_file("text_files/Football.txt")
word_document = rf.read_file("text_files/Technology.docx")
math_word_document = rf.read_file("text_files/Math.docx")

In [ ]:
# Summarize the article
bullet_points = ts.summarize(math_word_document)
print(bullet_points)
print(len(bullet_points))

In [ ]:
# Generate a general header
header = he.generate_slide_deck_title(bullet_points, 1)
print(header)

In [ ]:
# Generate an image using the general header using OpenAI's DALL-E
image = ig.generate_image_from_text(header)
image_path = 'generated_image.jpg'
ig.save_image(image, image_path)

In [ ]:
# Create a new PowerPoint presentation

prs = Presentation("themes/dark.pptx")

# Function to add a new slide
def add_new_slide(prs, header, image_path=None):
    slide = prs.slides.add_slide(prs.slide_layouts[6])
    title = slide.shapes.title
    title.text = header
    title.text_frame.paragraphs[0].runs[0].font.color.rgb = RGBColor(255, 255, 255)
    title.text_frame.paragraphs[0].runs[0].font.size = Pt(40)

    # Add a text box for bullet points
    left = Inches(0.2)
    top = Inches(1.5)
    width = Inches(6.7)
    height = Inches(2.5)
    textbox = slide.shapes.add_textbox(left, top, width, height)
    text_frame = textbox.text_frame
    text_frame.word_wrap = True

    # Add the generated image to the slide
    if image_path:
        img_left = Inches(7)
        img_top = Inches(2)
        img_height = Inches(2.5)
        img_width = Inches(2.5)
        slide.shapes.add_picture(image_path, img_left, img_top, height=img_height, width=img_width)

    return slide, text_frame

# Function to add dot bullets
def add_dot_bullet(paragraph):
    pPr = paragraph._p.get_or_add_pPr()
    buChar = OxmlElement('a:buChar')
    buChar.set('char', '•')
    pPr.insert(0, buChar)

# Process bullet points in chunks of 3 (3 bullet points per slide)
chunk_size = 3
for i in range(0, len(bullet_points), chunk_size):
    chunk = bullet_points[i:i+chunk_size]
    header = he.generate_slide_deck_title(chunk, 2)
      
    # # Generate an image for the current chunk using OpenAI's DALL-E
    # image = ig.generate_image_from_text(header)
    # image_path = f'generated_image_{i//chunk_size}.jpg'
    # ig.save_image(image, image_path)

 # Generate multiple image URLs for the current chunk
    image_urls = ss.search_google_images(header)
    image = None
    image_path = None

    # Try each URL until a valid image is found
    for image_url in image_urls:
        print(f"Trying image URL: {image_url}")
        image = ss.download_image(image_url)
        if image:
            image_path = f'generated_image_{i//chunk_size}.jpg'
            image_path = ss.save_image(image, image_path)
            break
    
    # Add a new slide for each chunk with the generated image
    slide, text_frame = add_new_slide(prs, header, image_path)
    
    # Add bullet points with smaller font size and dot bullet
    for point in chunk:
        if point.strip():  # Check if the point is not empty
            p = text_frame.add_paragraph()
            p.text = ' ' + point  # Add space after the bullet
            p.level = 0
            p.font.size = Pt(16)
            p.font.color.rgb = RGBColor(0, 0, 0)
            p.space_after = Pt(14)  # Add space after each point for better readability
            add_dot_bullet(p)  # Add dot bullet for each point
            
            # Add an empty paragraph for spacing
            empty_p = text_frame.add_paragraph()
            empty_p.text = ''
            empty_p.level = 0
            empty_p.font.size = Pt(18)
            empty_p.space_after = Pt(14)  # Add space after each point for better readability

# Save the presentation
prs.save('ppt.pptx')
print("Presentation saved successfully!")